In [ ]:
# pip install geopandas shapely

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

## 대여소별 가장 가까운 대여소 거리 구하기

In [1]:
# CSV 파일 경로
file_path = "/Users/minjikim/Desktop/sesac/프로젝트 1/contents/seoul_bicycle_masterinfo.csv"

# CSV 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
print(df.head())  # 상위 5개 행 확인

NameError: name 'pd' is not defined

In [ ]:
df["대여소_ID"]

0        ST-999
1        ST-998
2        ST-997
3        ST-996
4        ST-995
         ...   
3390    ST-1002
3391    ST-1001
3392    ST-1000
3393     ST-100
3394      ST-10
Name: 대여소_ID, Length: 3395, dtype: object

In [ ]:
df['location'] = list(zip(df['경도'], df['위도']))
df['point_location'] = df['location'].apply(lambda x: Point(x))
df['point_location'].head()

0                     POINT (0 0)
1                     POINT (0 0)
2     POINT (126.869598 37.53439)
3    POINT (126.850548 37.524334)
4    POINT (126.857323 37.510597)
Name: point_location, dtype: object

In [ ]:
# 1️⃣ point_location 이미 Point라고 가정
geometry = gpd.GeoSeries(df['point_location'])

# 2️⃣ GeoDataFrame 생성
seoul_stations = gpd.GeoDataFrame(df[['대여소_ID']].copy(), geometry=geometry)

# 3️⃣ 현재 좌표계 지정 (WGS84 위도/경도)
seoul_stations = seoul_stations.set_crs(epsg=4326)

# 4️⃣ 미터 단위 좌표계(Korea TM)로 변환
seoul_stations = seoul_stations.to_crs(epsg=5179)

# 5️⃣ 거리 계산 (벡터화)
dist_matrix = seoul_stations.geometry.apply(lambda x: seoul_stations.distance(x))
np.fill_diagonal(dist_matrix.values, np.nan)
seoul_stations['nearest_station_dist'] = dist_matrix.min(axis=1).round(2)

In [ ]:
seoul_stations

,대여소_ID,geometry,nearest_station_dist
0,ST-999,POINT (-5899757.108 17790114.866),0.00
1,ST-998,POINT (-5899757.108 17790114.866),0.00
2,ST-997,POINT (944303.993 1948528.325),218.83
3,ST-996,POINT (942613.203 1947424.115),260.70
4,ST-995,POINT (943201.45 1945895.954),162.59
...,...,...,...
3390,ST-1002,POINT (944914.111 1948026.093),137.72
3391,ST-1001,POINT (-5899757.108 17790114.866),0.00
3392,ST-1000,POINT (944038.679 1945866.202),250.53
3393,ST-100,POINT (962328.201 1948679.677),284.26


## 대여소별 일주일 대여건수 구하기

In [ ]:
file_path = "/Users/minjikim/Desktop/sesac/프로젝트 1/contents/tpss_bcycl_od_statnhm_20251011.csv"

# CSV 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
df

## 대여소기준 가까운 지하철역, 버스정류장과의 거리
#### 최은진 2025/10/17

In [3]:
# 파일 경로
bike_csv_path = 'data/seoul_bicycle_masterinfo.csv'
subway_csv_path = 'data/서울시 역사 마스터 정보.csv'
bus_csv_path = 'data/서울시 버스정류소 위치정보.csv'

# 컬럼명 설정

# 따릉이 대여소 파일
BIKE_LON_COL = '경도' 
BIKE_LAT_COL = '위도' 

# 지하철 역사 파일
SUBWAY_LON_COL = '경도' 
SUBWAY_LAT_COL = '위도'

# 버스 정류장 파일
BUS_LON_COL = 'X좌표' 
BUS_LAT_COL = 'Y좌표'

In [7]:
# CSV 파일을 GeoDataFrame으로 변환하는 함수 정의
def create_gdf(file_path, lon_col, lat_col):
    df = pd.read_csv(file_path, encoding='cp949')
    geometry = gpd.points_from_xy(df[lon_col], df[lat_col])
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    return gdf

# 데이터 로드 및 변환
bike_stations = create_gdf(bike_csv_path, BIKE_LON_COL, BIKE_LAT_COL)
subway_stations = create_gdf(subway_csv_path, SUBWAY_LON_COL, SUBWAY_LAT_COL)
bus_stations = create_gdf(bus_csv_path, BUS_LON_COL, BUS_LAT_COL)

# 데이터 로드 확인
if bike_stations is None or subway_stations is None or bus_stations is None:
    print("데이터 로드에 문제가 있습니다. 컬럼명을 다시 확인해주세요.")
    # exit() 
else:
    print(f"따릉이 대여소 수: {len(bike_stations)}")
    print(f"지하철 역사 수: {len(subway_stations)}")
    print(f"버스 정류장 수: {len(bus_stations)}")

따릉이 대여소 수: 3395
지하철 역사 수: 783
버스 정류장 수: 11290


In [14]:
CRS_METER = 'EPSG:5186' 

bike_stations = bike_stations.to_crs(CRS_METER) 

# 지하철, 버스 데이터 변환 (이상치 제거 전에 변환해야 함)
subway_stations_m = subway_stations.to_crs(CRS_METER) 
bus_stations_m = bus_stations.to_crs(CRS_METER)

## 이상치 제거

In [15]:
# 지하철역 WGS84 좌표계로 돌아가서 이상치 확인
subway_stations_wgs84 = subway_stations.to_crs(epsg=4326)

# 서울 범위 이외의 좌표를 가진 이상치 제거 (경도 126.5~127.5, 위도 37.0~38.0)
subway_stations_clean = subway_stations_wgs84[
    (subway_stations_wgs84[SUBWAY_LON_COL] > 126.5) & 
    (subway_stations_wgs84[SUBWAY_LON_COL] < 127.5) & 
    (subway_stations_wgs84[SUBWAY_LAT_COL] > 37.0) & 
    (subway_stations_wgs84[SUBWAY_LAT_COL] < 38.0)
].copy()

# 다시 미터 좌표계로 변환하여 사용
subway_stations_clean = subway_stations_clean.to_crs(CRS_METER)
print(f"이상치 제거 전 지하철역 수: {len(subway_stations)}")
print(f"이상치 제거 후 지하철역 수: {len(subway_stations_clean)}")

이상치 제거 전 지하철역 수: 783
이상치 제거 후 지하철역 수: 753


In [16]:
# 버스 정류장 이상치 확인
bus_stations_wgs84 = bus_stations.to_crs(epsg=4326)

# 서울 범위 이외의 좌표를 가진 이상치 제거 (경도 126.5~127.5, 위도 37.0~38.0)
bus_stations_clean = bus_stations_wgs84[
    (bus_stations_wgs84[BUS_LON_COL] > 126.5) & 
    (bus_stations_wgs84[BUS_LON_COL] < 127.5) & 
    (bus_stations_wgs84[BUS_LAT_COL] > 37.0) & 
    (bus_stations_wgs84[BUS_LAT_COL] < 38.0)
].copy()

# 다시 미터 좌표계로 변환하여 사용
bus_stations_clean = bus_stations_clean.to_crs(CRS_METER)
print(f"이상치 제거 전 버스 정류장 수: {len(bus_stations)}")
print(f"이상치 제거 후 버스 정류장 수: {len(bus_stations_clean)}")

이상치 제거 전 버스 정류장 수: 11290
이상치 제거 후 버스 정류장 수: 11290


In [17]:
# 거리 계산

# 지하철 거리 계산
nearest_subway_dist = []
for bike_station_geom in bike_stations.geometry:
    # 정제된 지하철 데이터 사용
    dist = subway_stations_clean.distance(bike_station_geom).min() 
    nearest_subway_dist.append(dist)

# 버스 거리 계산
nearest_bus_dist = []
for bike_station_geom in bike_stations.geometry:
    # 정제된 버스 데이터 사용
    dist = bus_stations_clean.distance(bike_station_geom).min()
    nearest_bus_dist.append(dist)

# 결과 저장 및 반올림
bike_stations['nearest_subway_dist_m'] = np.array(nearest_subway_dist).round(2)
bike_stations['nearest_bus_dist_m'] = np.array(nearest_bus_dist).round(2)
print("거리 계산 완.")

거리 계산 완.


In [18]:
MAX_DISTANCE_M = 10000 

# 지하철 거리 또는 버스 거리 중 하나라도 MAX_DISTANCE_M을 초과하는 행을 제거
bike_stations_clean = bike_stations[
    (bike_stations['nearest_subway_dist_m'] < MAX_DISTANCE_M) & 
    (bike_stations['nearest_bus_dist_m'] < MAX_DISTANCE_M)
].copy()

print(f"이상치 제거 전 대여소 수: {len(bike_stations)}")
print(f"이상치 제거 후 대여소 수: {len(bike_stations_clean)}")

이상치 제거 전 대여소 수: 3395
이상치 제거 후 대여소 수: 3318


# 이상치 제거 후 최소 거리 계산

In [19]:
# 최소 거리 계산
bike_stations_clean = bike_stations_clean.to_crs(CRS_METER)
subway_stations_clean = subway_stations_clean.to_crs(CRS_METER)
bus_stations_clean = bus_stations_clean.to_crs(CRS_METER)

nearest_bus_dist = []
nearest_subway_dist = []

for bike_station_geom in bike_stations_clean.geometry:
    dist = bus_stations_clean.distance(bike_station_geom).min()
    nearest_bus_dist.append(dist)

for bike_station_geom in bike_stations_clean.geometry:
    dist = subway_stations_clean.distance(bike_station_geom).min()
    nearest_subway_dist.append(dist)

# 소수점 2자리까지
bike_stations_clean['nearest_bus_dist_m'] = np.array(nearest_bus_dist).round(2)
bike_stations_clean['nearest_subway_dist_m'] = np.array(nearest_subway_dist).round(2)

# 결과 저장
print("### 지하철역, 버스 정류장 최소 거리 계산 완 ###")
print(bike_stations_clean[['대여소_ID', 'nearest_subway_dist_m', 'nearest_bus_dist_m']].head()) 

### 지하철역, 버스 정류장 최소 거리 계산 완 ###
   대여소_ID  nearest_subway_dist_m  nearest_bus_dist_m
2  ST-997                1011.88               24.27
3  ST-996                 504.22               63.21
4  ST-995                 777.29               65.25
5  ST-994                 560.82              102.10
6  ST-993                 401.02               97.40


In [20]:
BIKE_ID_COL = '대여소_ID' 

df = bike_stations_clean[[
    BIKE_ID_COL, 
    'nearest_subway_dist_m', 
    'nearest_bus_dist_m'
]].copy()
df

,대여소_ID,nearest_subway_dist_m,nearest_bus_dist_m
2,ST-997,1011.88,24.27
3,ST-996,504.22,63.21
4,ST-995,777.29,65.25
5,ST-994,560.82,102.10
6,ST-993,401.02,97.40
...,...,...,...
3389,ST-1003,1096.60,68.32
3390,ST-1002,611.71,92.05
3392,ST-1000,240.12,10.14
3393,ST-100,510.95,227.68


In [22]:
# CSV 파일로 저장
csv_path = 'data/seoul_bike_accessibility_features.csv'

# CSV 파일로 저장 (인덱스 제외, 인코딩: 한글 지원 cp949)
df.to_csv(csv_path, index=False, encoding='cp949')

print(f"'{csv_path}'에 저장되었습니다.")

'data/seoul_bike_accessibility_features.csv'에 저장되었습니다.
